In [1]:
from fastai.structured import *
from fastai.column_data import *

Note: The original training set is 5GB in size. To make the data more manageable and to reduce training times, i've limited the training set to the 2017 sales data. 

I will not be using the transactions data, as we don't have that data for the test set, and the holidays data, as the holiday effect should in theory already be captured in the training set sales data. 

In [2]:
PATH = '/home/kinzy-t/Documents/Structured Data/Favorita'
csv_names = ['train.csv','stores.csv','oil.csv','items.csv', 'test.csv']

In [3]:
#Load data
tables = [pd.read_csv(f'{csv}', low_memory=False) for csv in csv_names]
for t in tables:
    display(t.head())
    print('Rowcount:', len(t))

,Unnamed: 0,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,101688779,2017-01-01,25,99197,1.0,False
1,1,101688780,2017-01-01,25,103665,7.0,False
2,2,101688781,2017-01-01,25,105574,1.0,False
3,3,101688782,2017-01-01,25,105857,4.0,False
4,4,101688783,2017-01-01,25,106716,2.0,False


Rowcount: 23808261


,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


Rowcount: 54


,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


Rowcount: 1218


,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0
3,103520,GROCERY I,1028,0
4,103665,BREAD/BAKERY,2712,1


Rowcount: 4100


,id,date,store_nbr,item_nbr,onpromotion
0,125497040,2017-08-16,1,96995,False
1,125497041,2017-08-16,1,99197,False
2,125497042,2017-08-16,1,103501,False
3,125497043,2017-08-16,1,103520,False
4,125497044,2017-08-16,1,103665,False


Rowcount: 3370464


In [4]:
#Delete uneccesary column
tables[0] = tables[0].drop('Unnamed: 0', axis=1)
tables[0] = tables[0].drop('id', axis=1)

In [5]:
for t in tables:
    display(DataFrameSummary(t).summary())

,date,store_nbr,item_nbr,unit_sales,onpromotion
count,NaN,2.38083e+07,2.38083e+07,2.38083e+07,NaN
mean,NaN,27.9491,1.16041e+06,8.15754,NaN
std,NaN,16.22,579511,24.0645,NaN
min,NaN,1,96995,-10002,NaN
25%,NaN,13,687549,2,NaN
50%,NaN,28,1.17656e+06,4,NaN
75%,NaN,43,1.50154e+06,8,NaN
max,NaN,54,2.12711e+06,17146,NaN
counts,23808261,23808261,23808261,23808261,23808261
uniques,227,54,4018,116535,2


,store_nbr,city,state,type,cluster
count,54,NaN,NaN,NaN,54
mean,27.5,NaN,NaN,NaN,8.48148
std,15.7321,NaN,NaN,NaN,4.69339
min,1,NaN,NaN,NaN,1
25%,14.25,NaN,NaN,NaN,4
50%,27.5,NaN,NaN,NaN,8.5
75%,40.75,NaN,NaN,NaN,13
max,54,NaN,NaN,NaN,17
counts,54,54,54,54,54
uniques,54,22,16,5,17


,date,dcoilwtico
count,NaN,1175
mean,NaN,67.7144
std,NaN,25.6305
min,NaN,26.19
25%,NaN,46.405
50%,NaN,53.19
75%,NaN,95.66
max,NaN,110.62
counts,1218,1175
uniques,1218,998


,item_nbr,family,class,perishable
count,4100,NaN,4100,4100
mean,1.25144e+06,NaN,2169.65,0.240488
std,587687,NaN,1484.91,0.427432
min,96995,NaN,1002,0
25%,818111,NaN,1068,0
50%,1.3062e+06,NaN,2004,0
75%,1.90492e+06,NaN,2990.5,0
max,2.13424e+06,NaN,7780,1
counts,4100,4100,4100,4100
uniques,4100,33,337,2


,id,date,store_nbr,item_nbr,onpromotion
count,3.37046e+06,NaN,3.37046e+06,3.37046e+06,NaN
mean,1.27182e+08,NaN,27.5,1.2448e+06,NaN
std,972969,NaN,15.5858,589836,NaN
min,1.25497e+08,NaN,1,96995,NaN
25%,1.2634e+08,NaN,14,805321,NaN
50%,1.27182e+08,NaN,27.5,1.29466e+06,NaN
75%,1.28025e+08,NaN,41,1.73002e+06,NaN
max,1.28868e+08,NaN,54,2.13424e+06,NaN
counts,3370464,3370464,3370464,3370464,3370464
uniques,3370464,16,54,3901,2


The first we can do is to join the oil prices to the training sales data

In [6]:
tables[0] = pd.merge(tables[0], tables[2], on='date', how='left')
tables[0].head()

,date,store_nbr,item_nbr,unit_sales,onpromotion,dcoilwtico
0,2017-01-01,25,99197,1.0,False,NaN
1,2017-01-01,25,103665,7.0,False,NaN
2,2017-01-01,25,105574,1.0,False,NaN
3,2017-01-01,25,105857,4.0,False,NaN
4,2017-01-01,25,106716,2.0,False,NaN


Next we can impute the missing oil price data with the median value

In [7]:
tables[0]['dcoilwtico'] = tables[0]['dcoilwtico'].fillna(tables[0]['dcoilwtico'].median())
tables[0].head()

,date,store_nbr,item_nbr,unit_sales,onpromotion,dcoilwtico
0,2017-01-01,25,99197,1.0,False,49.22
1,2017-01-01,25,103665,7.0,False,49.22
2,2017-01-01,25,105574,1.0,False,49.22
3,2017-01-01,25,105857,4.0,False,49.22
4,2017-01-01,25,106716,2.0,False,49.22


Now we need to incorporate the perishable data from the items table. Model performance is evaluated using Normalized Weighted Root Mean Squared Logarithmic Error, with the weights determined by weather the item is perisbale or not.

In [8]:
tables[0] = pd.merge(tables[0], tables[3][['item_nbr', 'perishable']], on = 'item_nbr', how='left')
tables[0].head()

,date,store_nbr,item_nbr,unit_sales,onpromotion,dcoilwtico,perishable
0,2017-01-01,25,99197,1.0,False,49.22,0
1,2017-01-01,25,103665,7.0,False,49.22,1
2,2017-01-01,25,105574,1.0,False,49.22,0
3,2017-01-01,25,105857,4.0,False,49.22,0
4,2017-01-01,25,106716,2.0,False,49.22,0


In order to train the eventual neural network with time series data, we'll need to split the date column into categorical variables. We can use the fastai function`add_datepart`to do this:

In [9]:
add_datepart(tables[0], "date", drop=False);
tables[0].head()

,date,store_nbr,item_nbr,unit_sales,onpromotion,dcoilwtico,perishable,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,2017-01-01,25,99197,1.0,False,49.22,0,2017,1,52,1,6,1,False,True,False,True,False,True,1483228800
1,2017-01-01,25,103665,7.0,False,49.22,1,2017,1,52,1,6,1,False,True,False,True,False,True,1483228800
2,2017-01-01,25,105574,1.0,False,49.22,0,2017,1,52,1,6,1,False,True,False,True,False,True,1483228800
3,2017-01-01,25,105857,4.0,False,49.22,0,2017,1,52,1,6,1,False,True,False,True,False,True,1483228800
4,2017-01-01,25,106716,2.0,False,49.22,0,2017,1,52,1,6,1,False,True,False,True,False,True,1483228800


Now to prepare the test set:

In [11]:
tables[4] = pd.merge(tables[4], tables[2], on='date', how='left')
tables[4].head()

,id,date,store_nbr,item_nbr,onpromotion,dcoilwtico
0,125497040,2017-08-16,1,96995,False,46.8
1,125497041,2017-08-16,1,99197,False,46.8
2,125497042,2017-08-16,1,103501,False,46.8
3,125497043,2017-08-16,1,103520,False,46.8
4,125497044,2017-08-16,1,103665,False,46.8


Now we're going to split the training set into a training and cross validation set. As we're trying to predict future sales, we'll use the most recent training data as the cross validation set, and we'll use the exact same period of time as the test set. 

In [10]:
#Check latest data in training set
max(tables[0]['date'])

Timestamp('2017-08-15 00:00:00')

In [12]:
# list the dates in test set
tables[3]['date'].unique()

array(['2017-08-16', '2017-08-17', '2017-08-18', '2017-08-19', '2017-08-20', '2017-08-21', '2017-08-22',
       '2017-08-23', '2017-08-24', '2017-08-25', '2017-08-26', '2017-08-27', '2017-08-28', '2017-08-29',
       '2017-08-30', '2017-08-31'], dtype=object)

So we have 16 days in the test set, we'll use the last 16 days worth of data in the training set as the validation set:

In [13]:
val_idx = np.flatnonzero(
    (tables[0]['date']<=datetime.datetime(2017,8,15)) & (tables[0]['date']>=datetime.datetime(2017,7,30)))

In [21]:
tables[0] = tables[0].drop('date', axis=1);

In [22]:
cat_vars = ['store_nbr', 'item_nbr', 'onpromotion', 'perishable', 'Year', 'Month', 'Week', 'Day', 
            'Dayofweek', 'Dayofyear', 'Is_month_end', 'Is_month_start', 'Is_quarter_end', 
            'Is_quarter_start', 'Is_year_end', 'Is_year_start' ]

cont_vars = ['dcoilwtico', 'Elapsed']

dep = 'unit_sales'

In [23]:
for column in cat_vars:
    tables[0][column] = tables[0][column].astype('category').cat.as_ordered()
    
for column in cont_vars:
    tables[0][column] = tables[0][column].astype('float32')

In [29]:
df, y, nas, mapper = proc_df(tables[0], dep, do_scale=True)
yl = np.log(y)

/home/kinzy-t/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log
  


In [33]:
df_test, _, nas, mapper = proc_df(tables[3], dep, do_scale=True, skip_flds=['Id'],
                                 mapper=mapper, na_dict=nas)

In [31]:
#df.head(5)
proc_df

,store_nbr,item_nbr,onpromotion,dcoilwtico,perishable,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,25,2,1,-0.044452,1,1,1,34,1,7,1,1,2,1,2,1,2,-1.744974
1,25,5,1,-0.044452,2,1,1,34,1,7,1,1,2,1,2,1,2,-1.744974
2,25,6,1,-0.044452,1,1,1,34,1,7,1,1,2,1,2,1,2,-1.744974
3,25,12,1,-0.044452,1,1,1,34,1,7,1,1,2,1,2,1,2,-1.744974
4,25,13,1,-0.044452,1,1,1,34,1,7,1,1,2,1,2,1,2,-1.744974


In [ ]:
model = ColumnarModelData.from_data_frame(PATH, val_idx, df,  )

array([1, 2])

In [ ]:
def ln_y(a):
    return np.exp(a)

def nwrmsle(a,b):
    temp = ln_y(b)
    
    

Create validation set (making it the same date length as the training set)

In [19]:
test = pd.read_csv('test.csv')
test['date'].unique()

array(['2017-08-16', '2017-08-17', '2017-08-18', '2017-08-19', '2017-08-20', '2017-08-21', '2017-08-22',
       '2017-08-23', '2017-08-24', '2017-08-25', '2017-08-26', '2017-08-27', '2017-08-28', '2017-08-29',
       '2017-08-30', '2017-08-31'], dtype=object)